In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.data_preprocessing import load_data, preprocess_data
import os
import numpy as np

save_data = "save_data_2018/"


cicids2018_path_v1 = 'data/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv'
cicids2018_path_v2 = 'data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv'
merge_data = object
if(not os.path.isfile(save_data + "merge.csv")):
    # Optionally, remove duplicate rows
    data_2018_v1 = load_data(cicids2018_path_v1)
    data_2018_v2 = load_data(cicids2018_path_v2)

    print("Headers match. Proceeding with merge.")
    merged_df = pd.concat([data_2018_v1, data_2018_v2], axis=0, ignore_index=True)
    merged_df = merged_df.drop_duplicates()
    merged_df.to_csv(save_data + "merge.csv", index=False)
    
    merge_data = load_data(save_data + "merge.csv")
    
if(not os.path.isfile(save_data + "web_attacks.csv")):
    preprocess_data(merge_data, save_data)

In [2]:
df = pd.read_csv(save_data + 'web_attacks.csv')
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'Benign' else 1)


normal_df = df[df['Label'] == 0]
attack_df = df[df['Label'] != 0]

num_attack = len(attack_df)

num_normal = 2 * num_attack

In [3]:
if num_normal > len(normal_df):
    raise ValueError("num_normal exceeds the number of rows in normal_df.")
normal_df = normal_df.sample(n=num_normal, random_state=42)

# Ensure 'Label' column exists
if 'Label' not in normal_df.columns or 'Label' not in attack_df.columns:
    raise ValueError("Both DataFrames must have a 'Label' column.")

# Combine and shuffle the dataset
balanced_df = pd.concat([normal_df, attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the balanced dataset
if 'Label' not in balanced_df.columns:
    raise ValueError("The 'Label' column is missing from the balanced DataFrame.")
train_df, temp_df = train_test_split(balanced_df, test_size=0.4, random_state=42, stratify=balanced_df['Label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['Label'])

# Ensure enough data for balancing
train_normal_df = train_df[train_df['Label'] == 0]
train_attack_df = train_df[train_df['Label'] == 1]

if len(train_attack_df) == 0:
    raise ValueError("There are no attack instances in the training set.")

num_train_attack = len(train_attack_df)
num_train_normal = 2 * num_train_attack

# Ensure we don't try to sample more rows than are available
if len(train_normal_df) < num_train_normal:
    num_train_normal = len(train_normal_df)  # Adjust to available number of rows

train_normal_df = train_normal_df.sample(n=num_train_normal, random_state=42)
train_df = pd.concat([train_normal_df, train_attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
# Print the count of each class in the training, validation, and testing sets
for name, df in zip(["Training", "Validation", "Testing"], [train_df, val_df, test_df]):
    unique, counts = np.unique(df['Label'], return_counts=True)
    print(f"{name} set class distribution:", dict(zip(unique, counts)))

Training set class distribution: {0: 1113, 1: 557}
Validation set class distribution: {0: 371, 1: 186}
Testing set class distribution: {0: 372, 1: 185}


In [5]:
# Save the train, validation, and test sets to CSV files
train_df.to_csv(save_data +  'train_set.csv', index=False)
val_df.to_csv(save_data +  'val_set.csv', index=False)
test_df.to_csv(save_data + 'test_set.csv', index=False)

In [6]:
# Define feature columns (excluding the label column)
feature_cols = [col for col in df.columns if col != 'Label']

# Extract features and labels for training, validation, and testing sets
X_train = train_df[feature_cols]
y_train = train_df['Label']
X_val = val_df[feature_cols]
y_val = val_df['Label']
X_test = test_df[feature_cols]
y_test = test_df['Label']

In [7]:
# Define classifiers
from models.decision_tree import train_decision_tree
from models.random_forest import train_random_forest
from models.lda import train_lda
from models.naive_bayes import train_naive_bayes
from models.logistic_regression import train_logistic_regression
from models.knn import train_knn
from models.svm import train_svm
from models.extra_trees import train_extra_trees
from models.bagging import train_bagging
from models.mlp import train_mlp

classifiers = {
    'Decision Tree': train_decision_tree,
    'Random Forest': train_random_forest,
    'Linear Discriminant Analysis': train_lda,
    'Naive Bayes': train_naive_bayes,
    'Logistic Regression': train_logistic_regression,
    'K-Nearest Neighbors': train_knn,
    'Support Vector Machine': train_svm,
    'Extra Trees Classifier': train_extra_trees,
    'Bagging Classifier': train_bagging,
    'Multi-layer Perceptron': train_mlp
}

In [8]:
def model_10Classifier():
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1 = train_function(X_train, y_train, X_val, y_val, X_test, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            "Test Accuracy": test_acc,
            "Test Precision": test_pre,
            "Test Recall": test_rec,
            "Test F1 Score": test_f1,
        })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(save_data + '10Classsifer_results_2018.csv', index=False)
    
model_10Classifier()

Decision Tree Validation - Accuracy: 0.9892280071813285 Precision: 0.9736842105263158 Recall: 0.9946236559139785 F1 Score: 0.9840425531914894
Random Forest Validation - Accuracy: 0.9946140035906643 Precision: 0.9893048128342246 Recall: 0.9946236559139785 F1 Score: 0.9919571045576407
LDA Validation - Accuracy: 0.9443447037701975 Precision: 0.8817733990147784 Recall: 0.9623655913978495 F1 Score: 0.9203084832904884
Naive Bayes Validation - Accuracy: 0.7091561938958707 Precision: 0.536144578313253 Recall: 0.956989247311828 F1 Score: 0.6872586872586872


c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Validation - Accuracy: 0.8186714542190305 Precision: 0.84 Recall: 0.5645161290322581 F1 Score: 0.6752411575562701
KNN Validation - Accuracy: 0.9587073608617595 Precision: 0.9095477386934674 Recall: 0.9731182795698925 F1 Score: 0.9402597402597402
SVM Validation - Accuracy: 0.7917414721723519 Precision: 0.9861111111111112 Recall: 0.3817204301075269 F1 Score: 0.5503875968992248
Extra Trees Validation - Accuracy: 0.9964093357271095 Precision: 0.9893617021276596 Recall: 1.0 F1 Score: 0.9946524064171123
MLP Validation - Accuracy: 0.8922800718132855 Precision: 0.8387096774193549 Recall: 0.8387096774193549 F1 Score: 0.8387096774193549


In [9]:
import pandas as pd
from sklearn.decomposition import PCA

# Assume the classifiers dictionary and train functions are already defined
# Define a function to apply PCA and train classifiers

def train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test, n_components=20):
    X_combined = np.concatenate((X_train, X_val, X_test), axis=0)
    y_combined = np.concatenate((y_train, y_val, y_test), axis=0)

    pca = PCA(n_components=n_components)
    X_combined_pca = pca.fit_transform(X_combined)

    # Bước 3: Tách lại dữ liệu thành các tập train, validation và test
    X_train_pca = X_combined_pca[:len(X_train)]
    X_val_pca = X_combined_pca[len(X_train):len(X_train)+len(X_val)]
    X_test_pca = X_combined_pca[len(X_train)+len(X_val):]

    # Kiểm tra kích thước của các tập dữ liệu sau khi PCA
    print("X_train_pca shape:", X_train_pca.shape)
    print("X_val_pca shape:", X_val_pca.shape)
    print("X_test_pca shape:", X_test_pca.shape)
    
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1 = train_function(X_train_pca, y_train, X_val_pca, y_val, X_test_pca, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1
        })
    
    return results

# Call the function and save results
pca_results = train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test)
pca_results_df = pd.DataFrame(pca_results)
pca_results_df.to_csv(save_data + 'pca_model_results_2018.csv', index=False)


X_train_pca shape: (1670, 20)
X_val_pca shape: (557, 20)
X_test_pca shape: (557, 20)
Decision Tree Validation - Accuracy: 0.9587073608617595 Precision: 0.9405405405405406 Recall: 0.9354838709677419 F1 Score: 0.9380053908355795
Random Forest Validation - Accuracy: 0.9694793536804309 Precision: 0.9567567567567568 Recall: 0.9516129032258065 F1 Score: 0.954177897574124
LDA Validation - Accuracy: 0.7809694793536804 Precision: 0.8902439024390244 Recall: 0.3924731182795699 F1 Score: 0.5447761194029851
Naive Bayes Validation - Accuracy: 0.39676840215439857 Precision: 0.33766233766233766 Recall: 0.8387096774193549 F1 Score: 0.48148148148148145


c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Validation - Accuracy: 0.8186714542190305 Precision: 0.8695652173913043 Recall: 0.5376344086021505 F1 Score: 0.6644518272425249
KNN Validation - Accuracy: 0.9317773788150808 Precision: 0.9021739130434783 Recall: 0.8924731182795699 F1 Score: 0.8972972972972973
SVM Validation - Accuracy: 0.7917414721723519 Precision: 0.9861111111111112 Recall: 0.3817204301075269 F1 Score: 0.5503875968992248
Extra Trees Validation - Accuracy: 0.9640933572710951 Precision: 0.9414893617021277 Recall: 0.9516129032258065 F1 Score: 0.946524064171123
MLP Validation - Accuracy: 0.8635547576301615 Precision: 0.9661016949152542 Recall: 0.6129032258064516 F1 Score: 0.75


In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 50
        encoding_dim2 = 30
        encoding_dim3 = int(np.sqrt(input_dim)) + 1

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.ReLU(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.ReLU()
        )
        # Lớp giải mã
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.ReLU(),
            nn.Linear(encoding_dim1, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, epochs=50, batch_size=256, alpha1=0.1):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Khởi tạo mô hình và các thành phần cần thiết
    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Chuyển đổi dữ liệu DataFrame thành NumPy array và tensor
    X_train_np = X_train.to_numpy()
    X_val_np = X_val.to_numpy()
    X_test_np = X_test.to_numpy()
    y_train_np = y_train.to_numpy()
    y_val_np = y_val.to_numpy()
    y_test_np = y_test.to_numpy()
    
    # Tạo TensorDataset cho training và validation
    train_dataset = TensorDataset(
        torch.tensor(X_train_np, dtype=torch.float32),
        torch.tensor(X_train_np, dtype=torch.float32)  # Sử dụng dữ liệu đầu vào làm mục tiêu cho autoencoder
    )
    val_dataset = TensorDataset(
        torch.tensor(X_val_np, dtype=torch.float32),
        torch.tensor(X_val_np, dtype=torch.float32)  # Sử dụng dữ liệu đầu vào làm mục tiêu cho autoencoder
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop
    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            encoded, outputs = autoencoder(inputs)
            loss = alpha1 * criterion(outputs, targets)  # Chỉ sử dụng MSE loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    # Encode dữ liệu
    autoencoder.eval()
    with torch.no_grad():
        X_train_enc, _ = autoencoder(torch.tensor(X_train_np, dtype=torch.float32).to(device))
        X_val_enc, _ = autoencoder(torch.tensor(X_val_np, dtype=torch.float32).to(device))
        X_test_enc, _ = autoencoder(torch.tensor(X_test_np, dtype=torch.float32).to(device))
    
    # Train và đánh giá classifiers
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1 = train_function(X_train_enc.cpu().numpy(), y_train, X_val_enc.cpu().numpy(), y_val, X_test_enc.cpu().numpy(), y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1
        })
    
    return results

# Gọi hàm và lưu kết quả
autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test)
autoencoder_results_df = pd.DataFrame(autoencoder_results)
autoencoder_results_df.to_csv(save_data + 'autoencoder_model_2018.csv', index=False)


Epoch [1/50], Loss: 11936803136658.285
Epoch [2/50], Loss: 11967147877522.285
Epoch [3/50], Loss: 11992866450870.857
Epoch [4/50], Loss: 12077963935744.0
Epoch [5/50], Loss: 11983782936576.0
Epoch [6/50], Loss: 11876845560978.285
Epoch [7/50], Loss: 11975827839853.715
Epoch [8/50], Loss: 12035234313947.428
Epoch [9/50], Loss: 11904329187328.0
Epoch [10/50], Loss: 12057901306441.143
Epoch [11/50], Loss: 12062069021549.715
Epoch [12/50], Loss: 12201278121106.285
Epoch [13/50], Loss: 11879714764507.428
Epoch [14/50], Loss: 12027118934893.715
Epoch [15/50], Loss: 12091767839597.715
Epoch [16/50], Loss: 11714842666130.285
Epoch [17/50], Loss: 12069116500845.715
Epoch [18/50], Loss: 11949041965933.715
Epoch [19/50], Loss: 11927130622244.572
Epoch [20/50], Loss: 11870434866907.428
Epoch [21/50], Loss: 11959366994212.572
Epoch [22/50], Loss: 11958180605366.857
Epoch [23/50], Loss: 12306422806235.428
Epoch [24/50], Loss: 11871630318445.715
Epoch [25/50], Loss: 11894611584146.285
Epoch [26/50], 

In [11]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 50
        encoding_dim2 = 30
        encoding_dim3 = int(np.sqrt(input_dim)) + 1

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.ReLU(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.ReLU()  # Đổi từ sigmoid thành ReLU để giữ giá trị dương
        )
        # Lớp giải mã
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.ReLU(),
            nn.Linear(encoding_dim1, input_dim),
            nn.Sigmoid()  # Giữ sigmoid cho đầu ra
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

class BinaryMagnetLoss(nn.Module):
    def __init__(self, alpha=7.18, epsilon=1e-6):
        super(BinaryMagnetLoss, self).__init__()
        self.alpha = alpha
        self.epsilon = epsilon

    def forward(self, embeddings, labels):
        _min_float = 1e-6
        embeddings = embeddings.float()
        batch_size = embeddings.size(0)

        unique_labels = labels.unique()
        batch_clusters = {label.item(): (labels == label).nonzero(as_tuple=True)[0] for label in unique_labels}

        num_instances = 0.0
        stdev = torch.zeros(1).to(embeddings.device)
        c_means = torch.stack([torch.mean(embeddings[batch_clusters[label.item()]], dim=0) for label in unique_labels]).to(embeddings.device)

        for label in unique_labels:
            cluster_indices = batch_clusters[label.item()]
            for i in cluster_indices:
                stdev += (embeddings[i] - c_means[unique_labels.tolist().index(label)]).norm(p=2).pow(2)
                num_instances += 1.0

        stdev = stdev / (num_instances - 1.0)
        stdev = -2.0 * (stdev + self.epsilon)  # Add epsilon to avoid division by zero

        loss = torch.zeros(1).to(embeddings.device)
        denom = [torch.zeros(1).to(embeddings.device) for _ in range(batch_size)]

        for label in unique_labels:
            cluster_indices = batch_clusters[label.item()]
            for i in cluster_indices:
                for other_label in unique_labels:
                    if other_label != label:
                        denom[i] += ((embeddings[i] - c_means[unique_labels.tolist().index(other_label)]).norm().pow(2) / stdev).exp()

                # Avoid log(0) by clamping values
                loss_term = (((embeddings[i] - c_means[unique_labels.tolist().index(label)]).norm().pow(2) / stdev - self.alpha).exp() / (denom[i] + self.epsilon)).log().clamp(max=0.0)
                if torch.isnan(loss_term) or torch.isinf(loss_term):
                    continue  # Skip this term if it's invalid
                loss -= loss_term

        loss /= num_instances
        return loss


# Custom Loss Function combining Cross-Entropy and Binary Magnet Loss
def custom_loss(y_true, y_pred, embeddings, alpha1=1.0, alpha2=1.0):
    ce_loss = F.cross_entropy(y_pred, y_true.long())  # Tính toán Cross-Entropy Loss
    magnet_loss_fn = BinaryMagnetLoss()
    magnet_loss_value = magnet_loss_fn(embeddings, y_true)
    return alpha1 * ce_loss + alpha2 * magnet_loss_value

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data, epochs=10, batch_size=256, alpha1=1.0, alpha2=1.0):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    X_train_np = X_train.to_numpy()
    X_val_np = X_val.to_numpy()
    y_train_np = y_train.to_numpy()
    y_val_np = y_val.to_numpy()

    train_dataset = TensorDataset(
        torch.tensor(X_train_np, dtype=torch.float32),
        torch.tensor(y_train_np, dtype=torch.float32)
    )
    val_dataset = TensorDataset(
        torch.tensor(X_val_np, dtype=torch.float32),
        torch.tensor(y_val_np, dtype=torch.float32)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs, encoded = autoencoder(inputs)
            loss = custom_loss(labels, outputs, encoded, alpha1=alpha1, alpha2=alpha2)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    autoencoder.eval()
    with torch.no_grad():
        X_train_enc = autoencoder.encoder(torch.tensor(X_train_np, dtype=torch.float32).to(device)).cpu().numpy()
        X_val_enc = autoencoder.encoder(torch.tensor(X_val_np, dtype=torch.float32).to(device)).cpu().numpy()
        X_test_enc = autoencoder.encoder(torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)).cpu().numpy()

    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1 = train_function(X_train_enc, y_train, X_val_enc, y_val, X_test_enc, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1
        })

    return results

# Gọi hàm và lưu kết quả
autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data)
autoencoder_results_df = pd.DataFrame(autoencoder_results)
autoencoder_results_df.to_csv(save_data + 'autoencoder_Binary_Magnet_Loss_results_2018.csv', index=False)


Epoch [1/10], Loss: 11.100731713431221
Epoch [2/10], Loss: 10.866159439086914
Epoch [3/10], Loss: 10.621945108686175
Epoch [4/10], Loss: 10.33474336351667
Epoch [5/10], Loss: 10.14861365727016
Epoch [6/10], Loss: 9.964658873421806
Epoch [7/10], Loss: 9.89177485874721
Epoch [8/10], Loss: 9.700197628566197
Epoch [9/10], Loss: 9.601284572056361
Epoch [10/10], Loss: 9.572335243225098
Decision Tree Validation - Accuracy: 0.9353680430879713 Precision: 0.936046511627907 Recall: 0.8655913978494624 F1 Score: 0.8994413407821229
Random Forest Validation - Accuracy: 0.947935368043088 Precision: 0.9289617486338798 Recall: 0.9139784946236559 F1 Score: 0.9214092140921409
LDA Validation - Accuracy: 0.8096947935368043 Precision: 0.8846153846153846 Recall: 0.4946236559139785 F1 Score: 0.6344827586206897
Naive Bayes Validation - Accuracy: 0.3644524236983842 Precision: 0.32857142857142857 Recall: 0.8655913978494624 F1 Score: 0.47633136094674555
Logistic Regression Validation - Accuracy: 0.8096947935368043